In questo notebook definisco le funzioni necessarie a:

    1. Creare un grafo
    2. Ottenere le varie misure (diametro, betwennes, numero community etc) dal grafo

num nodi, num link, diametro, num commuty, num elementi biggest comm, average degree, plot degree vs betwennes (estrai coefficiente correlazione e plottalo in funzione del tempo)

In [ ]:
!pip install igraph

In [ ]:
import pandas as pd
import numpy as np
import glob
import re

In [ ]:
from collections import Counter

In [ ]:

import networkx as nx
import matplotlib.pyplot as plt
import igraph as ig
import community
import community.community_louvain as community_louvain
import os


In [ ]:
#ritorna il network creato partendo da un df pulito
def create_network_from_dataframe(df):
   G = nx.from_pandas_edgelist(df, source='White', target='Black')
   return G

In [ ]:
def network_diameter(G):
   return nx.diameter(G)

In [ ]:
def central_cluster_of_network(G):
   largest_cc = max(nx.connected_components(G), key=len)
   cc = G.subgraph(largest_cc).copy()
   return cc


In [ ]:
 #questa ritorna 2 liste: la lista delle degree dei giocatori e la lista dei nomi dei giocatori
def degree_list(G):
   degree = dict(G.degree())
   names_list=list(degree.keys())
   degree_list=list(degree.values())
   return degree_list,names_list

In [ ]:
#questa prende il network e la bool che dice se vogliamo la betwenness normalizzata o no e ritorna 2 liste:
#la lista delle betwenness e la lista dei nomi dei giocatori

def betweenness_list(G,normalized):
   bet=dict(nx.betweenness_centrality(G,normalized=normalized))
   names_list=list(bet.keys())
   bet_list=list(bet.values())
   return bet_list,names_list

In [ ]:
#prende il netwoork e bool che falgga normalizzazione e ritorna media della betwennes
def average_betwennes(G, normalized):
  bet=dict(nx.betweenness_centrality(G,normalized=normalized))
  bet_list=list(bet.values())
  bet_avg= sum(bet_list)/len(bet_list)
  return bet_avg

In [ ]:
 #prende il netwoork ritorna media  delle degree dei giocatori
def average_degree(G):
   degree = dict(G.degree())
   degree_list=list(degree.values())
   deg_avg = sum(degree_list)/len(degree_list)
   return deg_avg

In [ ]:
#Prende in input un networkx graph e ritorna le community trovate con l'algoritmo louvain

def CommunityCounting(G):
  partition = community_louvain.best_partition(G, random_state=42)
  community_labels = {node: community_id for node, community_id in partition.items()}
  num_communities = max(community_labels.values()) + 1
  return community_labels, num_communities, G.number_of_nodes()

In [ ]:
def MostPopulatedCommunity(G):
  community_labels, num_communities, nodes = CommunityCounting(G)
  num_of_items = Counter(community_labels.values())
  num_of_items = dict(num_of_items)
  max_comm = max(num_of_items, key=num_of_items.get)
  nitems_of_max_community=max(num_of_items.values())
  percentage_of_max_comm_items = nitems_of_max_community/(nodes)
  return (num_communities, max_comm, nitems_of_max_community, percentage_of_max_comm_items)


In [ ]:
#fa scatterplot e printa dato il df di partenza, le due colonne che saranno x e y,
#la colonna dei labels e quella dei colori.
#Si downloada direttamente, non si salva

def scatterplot(df,feature1,feature2,labels,color):
   fig = px.scatter(data_frame=df, x=feature1, y=feature2, hover_name=labels, color=color)
   fig.show()

In [ ]:
#plotta e salva l'istogramma di un'osservabile coi nomi in verticale e.g. degree dei primi 50 giocatori
def plot_histogram_with_vertical_labels(Names_x_axis,observable):
   plt.bar(Names_x_axis,observable)
   plt.xticks(rotation='vertical')
   plt.savefig('./histogram_with_vertical_labels.png')
   plt.show()

In [ ]:
#ritorna il coef.angolare, l'intercetta e il coef. di correlazione tra due liste x e y

def linear_regression(x,y):
   result = linregress(x=x,y=y)
   return result.slope,result.intercept,result.rvalue

Qui inizia il processo vero e proprio. La cella immediatamente qui sotto serve a cercare tutti i file .csv nella cartella e salvarsi i nomi, ma volendo si può fare anche a mano se sono pochi.


In [ ]:
file_names = []
for file in glob.glob('./*.csv'):
  file_names.append(file)

Questa funzione prende in input una lista di percorsi e un dizionario, loopa sui file e fa tutta l'analisi, restituendo un dizionario.
Quelli che non hanno funzionato sono
['./chess_data_2005.csv',
 './chess_data_1990.csv',
 './chess_data_1995.csv',
 ]



In [ ]:
def analisi(file_names, dizionario):

  for file in file_names:
      try:
        df = pd.read_csv(file, low_memory= False)
        print('Processando file:', file)

        ''' estraggo l'anno dal nome del csv
            sfruttando il fatto che sono chess_data_1234.csv.
            In questo modo, più giù, posso creare una voce anno nel dizionario
            e avere tutti i valori di quell'anno'''

        anno = re.match(r"(.+?)_(\d+).csv", file).group(2)
        graph = create_network_from_dataframe(df)
        G = central_cluster_of_network(graph)
        num_nodes = G.number_of_nodes()
        num_edges = G.number_of_edges()
        diam = (network_diameter(G))
        avg_bet = (average_betwennes(G, normalized = True))
        avg_deg = (average_degree(G))
        num_comm, max_comm, n_items, percentage = MostPopulatedCommunity(G)
        dizionario[anno] = {  'max_elements': max_comm,
                              'num_comm': num_comm,
                              'n_items': n_items,
                              'percentage': percentage,
                              'avg_deg': avg_deg,
                              'avg_bet': avg_bet,
                              'diameter': diam,
                              'nodes': num_nodes,
                              'edges': num_edges,

                            }

      except Exception as e:
        print('Non ha funzionato:' , file)
  return(dizionario)

In [ ]:
comm_dict = {}
analisi(file_names, comm_dict)

In [ ]:
df = pd.DataFrame.from_dict(comm_dixt)
df.to_csv('dati_terza.csv')

In [ ]:
comm_dict


{'2010': {'max_elements': 2,
  'num_comm': 21,
  'n_items': 120,
  'percentage': 0.10118043844856661,
  'avg_deg': 5.942664418212479,
  'avg_bet': 0.004537599348232893,
  'diameter': 18,
  'nodes': 1186,
  'edges': 3524}}

In [ ]:
# DEBUG: qui provo a far calcolare alcune cose. Non hai informazioni su quale funzione dà errore
#ma sai quali file sono problematici

error_files = []
for file in file_names:
    try:
        df = pd.read_csv(file, low_memory= False)
        G2 = create_network_from_dataframe(df)
        G = central_cluster_of_network(G2)
        diameters.append(network_diameter(G))
    except Exception as e:
        error_files.append(file)